In [1]:
import json
import time

from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from tqdm import tqdm

In [2]:
base_url = "https://trustpilot.com"

In [3]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [4]:
#fetching sub-categories urls:

data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class': 'subCategory___BRUDy'}):
    name = category.find('h3', {'class': 'subCategoryHeader___36ykD'}).text
    name = name.strip()
    data[name] = {}
    
    sub_categories = category.find('div', {'class': 'subCategoryList___r67Qj'})
    for sub_category in sub_categories.findAll('div', {'class': 'subCategoryItem___3ksKz'}):
        sub_category_name = sub_category.find('a', {'class': 'navigation___2Efid'}).text 
        sub_category_uri = sub_category.find('a', {'class': 'navigation___2Efid'})['href'] 
        data[name][sub_category_name] = sub_category_uri

In [5]:
len(data)

22

In [6]:
# fetch company urls referenced in a given subcategory:

def extract_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('(//div[@class="businessUnitCardsContainer___Qhix1"])/a')
    urls = [a.get_attribute('href') for a in a_list]
    dedup_urls = list(set(urls))
    return dedup_urls

In [7]:
# if a pagination exists on the page:

def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        return True, button
    except NoSuchElementException:
        return False, None

In [8]:
# initializing Selenium with a headless firefoxdriver:

options = Options()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

driver = webdriver.Firefox(executable_path=r'driver\geckodriver.exe', options=options)

timeout = 5

In [ ]:
# scraping

company_urls = {}
for category in tqdm(data):
    for sub_category in tqdm(data[category], leave=False):
        company_urls[sub_category] = []
        print("Category is: ", category)
        print("sub-category is: ", sub_category)
        url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
        driver.get(url)
        print("URL is: ", url)
        try:
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'wrapper___2rOTx'))
            
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
    
        next_page = True
        c = 1
        while next_page:
            extracted_company_urls = extract_company_urls_form_page()
            print("company url is:",extracted_company_urls)
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            if next_page:
                c += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all" + f'&page={c}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'wrapper___2rOTx'))
                    # wrapper___2rOTx
                    WebDriverWait(driver, timeout).until(element_present)
                except:
                    pass

  0%|                                                  | 0/6 [00:00<?, ?it/s]

Category is:  Animals & Pets
sub-category is:  Animal Health


URL is:  https://trustpilot.com/categories/animal_health?numberofreviews=0&timeperiod=0&status=all
company url is: ['https://www.trustpilot.com/review/caninehealthcheck.com', 'https://www.trustpilot.com/review/www.vetapprovedrx.com', 'https://www.trustpilot.com/review/remedyhut.com', 'https://www.trustpilot.com/review/montereyanimalclinic.com', 'https://www.trustpilot.com/review/ocvh.com', 'https://www.trustpilot.com/review/pawprintgenetics.com', 'https://www.trustpilot.com/review/vetpetni.co.uk', 'https://www.trustpilot.com/review/stagingmybuddiesplace.azurewebsites.net', 'https://www.trustpilot.com/review/www.vetrxdirect.com']


 17%|███████                                   | 1/6 [00:07<00:37,  7.43s/it]

Category is:  Animals & Pets
sub-category is:  Animal Parks & Zoo
URL is:  https://trustpilot.com/categories/animal_parks_zoo?numberofreviews=0&timeperiod=0&status=all


...

company url is: ['https://www.trustpilot.com/review/www.lexmark.com', 'https://www.trustpilot.com/review/www.zonealarm.com', 'https://www.trustpilot.com/review/www.acehardware.com', 'https://www.trustpilot.com/review/www.tomshardware.com', 'https://www.trustpilot.com/review/www.trialpay.com', 'https://www.trustpilot.com/review/www.zoho.com', 'https://www.trustpilot.com/review/www.murgee.com', 'https://www.trustpilot.com/review/righthelpdesk.com', 'https://www.trustpilot.com/review/ning.com', 'https://www.trustpilot.com/review/att-iphone-unlock.com', 'https://www.trustpilot.com/review/www.tabouf.com', 'https://www.trustpilot.com/review/www.squarespace.com', 'https://www.trustpilot.com/review/lifeproof.com', 'https://www.trustpilot.com/review/www.creative.com', 'https://www.trustpilot.com/review/shopcaseboy.com', 'https://www.trustpilot.com/review/usertesting.com', 'https://www.trustpilot.com/review/www.build.com', 'https://www.trustpilot.com/review/www.usbgeek.com', 'https://www.trustpi


 60%|████████████████████████▌                | 3/5 [10:36<07:43, 231.98s/it]

company url is: ['https://www.trustpilot.com/review/www.topazlabs.com', 'https://www.trustpilot.com/review/www.surveyjunkie.com', 'https://www.trustpilot.com/review/www.nolo.com', 'https://www.trustpilot.com/review/storagepartsdirect.com', 'https://www.trustpilot.com/review/ibuypower.com', 'https://www.trustpilot.com/review/spyfu.com', 'https://www.trustpilot.com/review/wowvendor.com', 'https://www.trustpilot.com/review/www.factoryoutletstore.com', 'https://www.trustpilot.com/review/blurspy.com', 'https://www.trustpilot.com/review/privateinternetaccess.com', 'https://www.trustpilot.com/review/www.republicwireless.com', 'https://www.trustpilot.com/review/site123.com', 'https://www.trustpilot.com/review/www.lowes.com', 'https://www.trustpilot.com/review/leprestore.com', 'https://www.trustpilot.com/review/totalav.com', 'https://www.trustpilot.com/review/taskrabbit.com', 'https://www.trustpilot.com/review/www.cablewholesale.com', 'https://www.trustpilot.com/review/www.play-asia.com', 'http

company url is: ['https://www.trustpilot.com/review/a1wordpressteam.com', 'https://www.trustpilot.com/review/fastsoftwares.com', 'https://www.trustpilot.com/review/shootproof.com', 'https://www.trustpilot.com/review/idshield.com', 'https://www.trustpilot.com/review/hostiso.com', 'https://www.trustpilot.com/review/getunlocked.com', 'https://www.trustpilot.com/review/logicmanager.com', 'https://www.trustpilot.com/review/primedhosting.com', 'https://www.trustpilot.com/review/www.poshighway.com', 'https://www.trustpilot.com/review/safeyangstore.com', 'https://www.trustpilot.com/review/rockwelltrading.com', 'https://www.trustpilot.com/review/gibdevelopments.com', 'https://www.trustpilot.com/review/slothhost.com', 'https://www.trustpilot.com/review/www.arieana.net', 'https://www.trustpilot.com/review/softorino.com', 'https://www.trustpilot.com/review/fxsvps.com', 'https://www.trustpilot.com/review/pixellu.com', 'https://www.trustpilot.com/review/titansvault.com', 'https://www.trustpilot.com/

company url is: ['https://www.trustpilot.com/review/www.simplybits.com', 'https://www.trustpilot.com/review/www.topazlabs.com', 'https://www.trustpilot.com/review/nordvpn.com', 'https://www.trustpilot.com/review/www.dashlane.com', 'https://www.trustpilot.com/review/www.joomla.org', 'https://www.trustpilot.com/review/clickherepublishing.com', 'https://www.trustpilot.com/review/tollfreeforwarding.com', 'https://www.trustpilot.com/review/forestracks.com', 'https://www.trustpilot.com/review/producerloops.com', 'https://www.trustpilot.com/review/www.resilio.com', 'https://www.trustpilot.com/review/www.reimageplus.com', 'https://www.trustpilot.com/review/llamacolombia.com', 'https://www.trustpilot.com/review/www.stormyhosting.com', 'https://www.trustpilot.com/review/keepcalling.com', 'https://www.trustpilot.com/review/skyroam.com', 'https://www.trustpilot.com/review/tawagpinas.com', 'https://www.trustpilot.com/review/artic-hosting.com', 'https://www.trustpilot.com/review/wideanglesoftware.co